In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
data_path = "/content/gdrive/MyDrive/Colab Notebooks/Lancaster University/Treatment/Data/"
usas_path = "/content/gdrive/MyDrive/Colab Notebooks/Lancaster University/USAS - SA/"
data_path_op = "/content/gdrive/MyDrive/Colab Notebooks/Lancaster University/USAS - SA/Output/"

In [ ]:
!pip install https://github.com/UCREL/pymusas-models/releases/download/en_dual_none_contextual-0.3.3/en_dual_none_contextual-0.3.3-py3-none-any.whl
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.0/902.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install colorama

In [ ]:
import spacy

# We exclude the following components as we do not need them.
nlp = spacy.load('en_core_web_sm', exclude=['parser', 'ner'])
# Load the English PyMUSAS rule-based tagger in a separate spaCy pipeline
english_tagger_pipeline = spacy.load('en_dual_none_contextual')
# Adds the English PyMUSAS rule-based tagger to the main spaCy pipeline
nlp.add_pipe('pymusas_rule_based_tagger', source=english_tagger_pipeline)

In [ ]:
import re
import pandas as pd
import numpy as np
from colorama import Fore, Back, Style

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Defining the tagging function
def tag_text(text):
  usas_eval_tags = {}
  usas_list = {}
  sentences = nltk.sent_tokenize(text)
  for i in range(len(sentences)):
    if(sentences[i]):
      # print(Fore.GREEN + Style.BRIGHT + "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      # print(Fore.CYAN + Style.BRIGHT + f"Sentence {i}:", sentences[i])
      # print(Fore.CYAN + Style.BRIGHT + "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      sent = nlp(sentences[i])
      for token in sent:
        tags = token._.pymusas_tags
        # print(Fore.CYAN + f'{token.text}:' + Fore.GREEN, tags)
        for tag in tags:
          usas_list[token.text] = token._.pymusas_tags
      usas_eval_tags[i] = usas_list
  return usas_eval_tags

In [ ]:
usas_emo_cls = pd.read_excel(usas_path + "usas_emo_classes.xlsx")
usas_emo_cls_dict  = dict(zip(usas_emo_cls.Tag, usas_emo_cls.Class))

In [ ]:
# Defining the emotion tag extraction function
def usas_emotion_tags(text):
  usas_eval_tags = {}
  sentences = nltk.sent_tokenize(text)
  for i in range(len(sentences)):
    if(sentences[i]):
      usas_list = {}
      sent = nlp(sentences[i])
      # print("\n")
      # print(Fore.MAGENTA + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      # print(Fore.BLUE + Style.BRIGHT + f"Sentence {i}:", sentences[i])
      for token in sent:
        tags = token._.pymusas_tags
        emo_tags = [x for x in tags if x.startswith('E')]
        if emo_tags:
          usas_list[token.text] = emo_tags
          # print(Fore.MAGENTA + f'{token.text}:' + Fore.GREEN, emo_tags)
      usas_eval_tags[sentences[i]] = usas_list
  return usas_eval_tags

In [ ]:
# Defining the sentiment classifier
def usas_sentiment_classifier(usas_emo_tag, positive, negative, neutral):
  sentiment_cls = {}
  for sent, emo_dict in usas_emo_tag.items():
    neg, pos, neu = 0, 0, 0
    if(emo_dict):
      for key, val_list in emo_dict.items():
        for val in val_list:
          if val.endswith('-'):
            neg+=1
            negative+=1
          elif  val.endswith('+'):
            pos+=1
            positive+=1
          else:
            neu+=1
            neutral+=1
    else:
      neu+=1
      neutral+=1
    if neg==max(pos, neg, neu):
          sentiment = 'negative'
          sentiment_cls[sent] = sentiment
    elif pos==max(pos, neg, neu):
          sentiment = 'positive'
          sentiment_cls[sent] = sentiment
    else:
          sentiment = 'neutral'
          sentiment_cls[sent] = sentiment
    # print(Fore.MAGENTA + Style.BRIGHT + "Sentence:", sent)
    # print(Fore.GREEN + Style.BRIGHT + "class:", sentiment)
    # print("\n")
    # print(Fore.GREEN + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  return sentiment_cls, positive, negative, neutral

In [ ]:
# Defining the probability estimation function for sentiment classification at the sentence-level
def sentence_sentiment_prob(usas_emo_tag):
  sentiment_cls = {}
  for sent, emo_dict in usas_emo_tag.items():
    neg, pos, neu = 0, 0, 0
    if(emo_dict):
      for key, val_list in emo_dict.items():
        for val in val_list:
          if val.endswith('-'):
            neg+=1
          elif val.endswith('+'):
            pos+=1
          else:
            neu+=1
    else:
      neu+=1
    sentiment = f'positive={pos/(pos+neg+neu)}, negative={neg/(pos+neg+neu)}, neutral={neu/(pos+neg+neu)}'
    sentiment_cls[sent] = sentiment
    # print(Fore.GREEN + Style.BRIGHT + "Sentence:", sent)
    # print(Fore.CYAN + Style.BRIGHT + "class:", sentiment)
    # print("\n")
    # print(Fore.CYAN + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  return sentiment_cls

In [ ]:
def text_sentiment_prob(text, pos, neg, neu):
  prob = f'positive={pos/(pos+neg+neu)}, negative={neg/(pos+neg+neu)}, neutral={neu/(pos+neg+neu)}'
  # print(Fore.CYAN+ text)
  # print(Fore.GREEN+ "----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  # print(prob)
  return prob

In [ ]:
# Defining the emotion extraction function
def usas_emotion_classifier(text):
  usas_eval_tags = {}
  sentences = nltk.sent_tokenize(text)
  for i in range(len(sentences)):
    if(sentences[i]):
      usas_list = {}
      sent = nlp(sentences[i])
      # print("\n")
      # print(Fore.CYAN + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      # print(Fore.CYAN + Style.BRIGHT + f"Sentence {i}:", sentences[i])
      for token in sent:
        tags = token._.pymusas_tags
        emo_tags = [x for x in tags if x.startswith('E')]
        emo_cls = [usas_emo_cls_dict[tag[:-1]]  if (tag[-2]=="+") or (tag[-2]=="-") else usas_emo_cls_dict[tag] for tag in emo_tags]
        if emo_tags:
          usas_list[token.text] = emo_cls
          # print(Fore.CYAN +  Style.BRIGHT + f'{token.text}:' + Fore.GREEN, emo_tags)
          # print(Fore.CYAN + Style.BRIGHT + 'class:' + Fore.RED, emo_cls)
      usas_eval_tags[sentences[i]] = usas_list
  return usas_eval_tags